In [1]:
import numpy as numpy
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import xarray as xr

In [17]:
submission_data = pd.read_csv("data/submission_data_Stochastic_Parrots.csv")
training_data = pd.read_csv("data/training_data.csv")
nwp = xr.load_dataset("data/hres_1day_south_scotland_202101_202306.nc")
wf_capacity = pd.read_csv("data/Central Scotland Wind Farms.csv")
gb_wind_farms = pd.read_csv("data/onshore_wind_farms.csv", encoding="cp1252")

In [3]:
training_data.head()

,dtm,wind_generation_MW,solar_generation_MW,total_generation_MW
0,2021-01-01T00:00:00Z,544.598,0.0,544.598
1,2021-01-01T00:30:00Z,534.354,0.0,534.354
2,2021-01-01T01:00:00Z,482.512,0.0,482.512
3,2021-01-01T01:30:00Z,426.260,0.0,426.260
4,2021-01-01T02:00:00Z,346.434,0.0,346.434


In [7]:
wf_capacity

,Name,Installed Capacity,Latitude,Longitude
0,Auchrobert Wind Farm,36,55.623900,-3.984170
1,Andershaw Wind Farm,35,55.530080,-3.799647
2,Black Law,134,55.777870,-3.707898
3,Black Law Extension Phase 1,69,55.777870,-3.707898
4,Braes of Doune,74,56.269829,-4.058936
5,Clyde Central,196,55.419488,-3.634978
6,Clyde North,198,55.468552,-3.595056
7,Clyde South,128,55.419488,-3.634978
8,Douglas West Dalquhandy,45,55.565100,-3.872850
9,Galawhistle Wind Farm,56,55.500000,-4.000000


### Step 1 - Disaggregate Over Wind Farm Sites

In [30]:
gb_wind_farms_dict = {
    "Auchrobert Wind Farm": "Auchrobert Wind Farm",
    "Harestanes": "Harestanes",
    "Black Law": "Black Law",
    "Andershaw": "Andershaw Wind Farm",
    "Black Law Extension Phase 1": "Black Law Extension - 1a",
    "Braes of Doune	": "Braes O'Doune",
    "Clyde Central": "Clyde Extension",
    "Dalquhandy": "Dalquhandy Windfarm",
    "Galawhistle Wind Farm": "Galawhistle",
    "Kype Muir": "Kype Muir Wind Farm",
    "Middle Muir": "Middle Muir Wind Farm",
    "Tullo": "Tullo Farm",
    "Tullo Wind Farm Ext": "Tullo Wind Farm South (Ext.)",
    "Whitelee 1": "Whitelee",
    "Whitelee 2": "Whitelee Extension"}

gb_wind_farms.loc[gb_wind_farms["Site Name"].isin(list(gb_wind_farms_dict.values()))]

,Ref ID,Operator (or Applicant),Site Name,Technology Type,Installed Capacity (MWelec),Turbine Capacity (MW),No. of Turbines,Height of Turbines (m),X-coordinate,Y-coordinate
368,3116,Scottish Power Renewables,Black Law,Wind Onshore,124.0,2.30,54.0,NaN,289500.0,653500.0
371,3119,Greencoat UK Wind,Braes O'Doune,Wind Onshore,72.0,2.00,36.0,NaN,272590.0,710500.0
674,3489,CRE Energy/ Scottish Power,Whitelee,Wind Onshore,322.0,2.30,140.0,NaN,256800.0,645435.0
784,4422,Eneco,Tullo Wind Farm South (Ext.),Wind Onshore,10.0,2.50,4.0,NaN,375388.0,770733.0
1243,4119,Scottish Power Renewables,Harestanes,Wind Onshore,136.0,2.00,68.0,NaN,300000.0,595000.0
1311,4189,Lantern Bidco,Dalquhandy Windfarm,Wind Onshore,45.0,2.00,15.0,150.0,214836.0,591232.0
1370,4253,Scottish Power Renewables,Black Law Extension - 1a,Wind Onshore,48.4,1.67,29.0,NaN,289500.0,653500.0
1449,4337,Falck Renewables,Auchrobert Wind Farm,Wind Onshore,36.0,2.50,12.0,NaN,275154.0,638500.0
1478,4368,Ventient (formerly Infinis),Galawhistle,Wind Onshore,66.0,3.00,22.0,NaN,275440.0,629090.0
1544,4440,Banks Renewables,Middle Muir Wind Farm,Wind Onshore,51.0,3.40,15.0,NaN,286178.0,625826.0


In [42]:
# nwp["ws100"] = pow(pow(nwp.u100,2) + pow(nwp.v100,2), 0.5)

def get_variable_from_netcfd(lon, lat, var):

    return (nwp[var].sel(longitude=lon, latitude = lat, method="nearest")
            .to_dataframe()
            .set_index("valid_time")[[var]]
            .resample("1800s")
            .interpolate())

In [43]:
lon, lat = wf_capacity.loc[0, ["Longitude", "Latitude"]]
var = "ws100"

get_variable_from_netcfd(lon, lat, var)

,ws100
valid_time,
2021-01-02 00:00:00,5.173418
2021-01-02 00:30:00,5.375004
2021-01-02 01:00:00,5.576590
2021-01-02 01:30:00,5.780569
2021-01-02 02:00:00,5.984549
...,...
2023-06-30 21:00:00,9.278943
2023-06-30 21:30:00,9.428005
2023-06-30 22:00:00,9.577068


### Step 2 - Resample to 30 Mins

In [29]:
list(gb_wind_farms_dict.values())

['Auchrobert Wind Farm',
 'Harestanes',
 'Black Law',
 'Andershaw Wind Farm',
 'Black Law Extension - 1a',
 "Braes O'Doune",
 'Clyde Extension',
 'Dalquhandy Windfarm',
 'Galawhistle',
 'Kype Muir Wind Farm',
 'Middle Muir Wind Farm',
 'Tullo Farm',
 'Tullo Wind Farm South (Ext.)',
 'Whitelee',
 'Whitelee Extension']